In [48]:
import pandas as pd
from functools import reduce
import os

#Data from: https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2017

year_suffix = [(2001, "B"), (2003,"C"), (2005, "D"), (2007, "E"), (2009, "F"), (2011, "G"), (2013, "H"), (2015, "I"),
                (2017, "J"), (2019, "K"), (2021, "L")]

#NOTE: PrescriptionMeds_df duplicates SEQN values which messes with inner join


selected_variables = [
    #Identifier
    "SEQN",

    # Cardiovascular Health
    "CDQ001", "CDQ006", "CDQ008", "CDQ010",
    
    # Illicit Drug Use
    "DUQ200", "DUQ210", "DUQ220Q", "DUQ220U", "DUQ240",
    "DUQ250", "DUQ260", "DUQ270Q", "DUQ270U", "DUQ280",
    "DUQ330", "DUQ340", "DUQ350Q", "DUQ350U", "DUQ380D",
    
    # Health Insurance
    "HIQ011", "HIQ032A", "HIQ032B", "HIQ210",
    
    # Medical Conditions
    "MCQ160b", "MCQ160c", "MCQ160d", "MCQ160e", "MCQ160f",
    "MCQ160m", "MCQ220", "MCQ160p", "MCQ160l",
    
    # Prescription Medications
    "RXQ510", "RXQ515", "RXQ520", "RXQ525G", "RXD530",
    
    # Weight History
    "WHD010", "WHD020", "WHQ030", "WHQ040", "WHD045",
    "WHD220", "WHD080A", "WHD080B", "WHD080C", "WHD080D",
    "WHD080P", "WHD080Q", "WHD080R", "WHD080S", "WHD080T"
]

def df_by_year_exp(char):
    file_paths = {
        "CardiovascularHealth": f"./data/CardiovascularHealth/CDQ_{char}.xpt",
        "IllicitDrugUse": f"./data/IllicitDrugUse/DUQ_{char}.xpt",
        "HealthInsurance": f"./data/HealthInsurance/HIQ_{char}.xpt",
        "MedicalConditions": f"./data/MedicalConditions/MCQ_{char}.xpt",
        # "PrescriptionMeds": f"./data/PrescriptionMeds/RXQ_RX_{char}.xpt",
        "PreventiveNSAID": f"./data/PreventativeNSAID/RXQASA_{char}.xpt",
        "WeightHistory": f"./data/WeightHistory/WHQ_{char}.xpt"
    }

    df_list = []

    for category, path in file_paths.items():
        if os.path.isfile(path):
            df = pd.read_sas(path)
            df_list.append(df)
        else:
            print(f"File not found: {path}")

    if not df_list:
        print("No files were loaded.")
        return None

    # Perform iterative inner join on "SEQN" column
    merged_df = reduce(lambda left, right: pd.merge(left, right, on="SEQN", how="inner"), df_list)
    filtered_df = merged_df[[col for col in selected_variables if col in merged_df.columns]]

    return filtered_df

df = df_by_year_exp("D")
df


File not found: ./data/PreventativeNSAID/RXQASA_D.xpt


,SEQN,CDQ001,CDQ006,CDQ008,CDQ010,DUQ200,DUQ210,DUQ220Q,DUQ220U,DUQ240,...,WHQ040,WHD220,WHD080A,WHD080B,WHD080C,WHD080D,WHD080P,WHD080Q,WHD080R,WHD080S
0,31131.0,2.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,2.0,...,2.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31151.0,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,2.0,...,2.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31153.0,1.0,NaN,2.0,2.0,1.0,15.0,15.0,4.0,1.0,...,2.0,45.0,10.0,11.0,12.0,13.0,NaN,NaN,NaN,NaN
3,31156.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,2.0,20.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN
4,31164.0,2.0,NaN,NaN,2.0,1.0,20.0,34.0,4.0,2.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,41447.0,2.0,NaN,NaN,2.0,1.0,21.0,24.0,4.0,2.0,...,2.0,50.0,10.0,11.0,12.0,13.0,NaN,NaN,NaN,NaN
1442,41450.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,2.0,...,2.0,50.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1443,41458.0,2.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,1.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,41462.0,2.0,NaN,NaN,2.0,1.0,19.0,20.0,4.0,2.0,...,2.0,6.0,10.0,NaN,12.0,13.0,NaN,NaN,NaN,NaN
